In [3]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas_datareader.data as web
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import warnings
import pandas as pd
from pandas import datetime
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
register_matplotlib_converters()
from time import time
warnings.filterwarnings("ignore")
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import pacf
from datetime import date
from datetime import datetime
import yfinance as yf

/var/folders/bk/zfnnqlb13tb44jdtl1wvzrn00000gn/T/ipykernel_4749/2708827875.py:10: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [5]:
#Funktion zur Berechnung der Trading-Gebühren 
def trading_fee(endowment):
    fee = endowment*(order_fee/100)
    
    #Prüfung, ob Mindest- oder Maximalgebühr des ausgewählten Brokers greift
    if fee <= min_:
        fee = min_
        
    if max_.isnumeric():
        if fee >= max_:
            fee = max_
        
    return fee

In [6]:
#Auswahl des Trading-Brokers
broker_auswahl = pd.read_csv('Trading_Broker.csv')

broker = str(input("Gewünschter Online-Broker: "))
print(broker)

i = broker_auswahl[broker_auswahl['Anbieter'] == broker].index
print(i[0])

print(broker_auswahl.at[i[0], "Gebühr"])
order_fee = broker_auswahl.at[i[0], "Gebühr"]
min_ = broker_auswahl.at[i[0], "Minimum"]
max_= broker_auswahl.at[i[0], "Maximum"]

Gewünschter Online-Broker:  IG


IG
1
0.05


In [7]:
#Nutzerentscheidung, ob Short-Selling aktiviert werden soll -> Eingabe: "Yes" oder "No"
short = False
answer = str(input("Include short-selling?: "))
print(answer)
if answer == "Yes":
    short = True

Include short-selling?:  Yes


Yes


In [8]:
#Funktion zum Download der relevanten Daten
def get_data(asset, start, end):
    ticker = asset
    data = yf.download(ticker, start=start, end=end)

    # calculate returns and format df
    data["Returns"] = 100 * data["Adj Close"].pct_change()
    data["Log. Returns"] = np.log(1+(data["Returns"]/100))
    data = data.dropna()
    data["Date"] = pd.to_datetime(data.index)

    data.reset_index(drop=True,inplace=True)
    return data

In [9]:
#Funktion, welche das betrachtete Zeitfenster in den Test-Datensatz (alle Tage aus der aktuellste) und den "Trainings"-Datensatz aufteilt (mit diesem Wert wird die Vorhersagegüte und die Endowment-Entwicklung berechnet)
def split_one_day(data):
    df_train, df_test = data.iloc[:-1], data.iloc[-1:]
    return df_train, df_test

In [10]:
#Funktion zur Durchführung der statistischen Auswertung der PACF und der optimalen Lag-Anzahl 
def statistic_tests(test_data, number_lags):
    trading = True
    AIC = []
    BIC = []
    a, ci = pacf(test_data, nlags=number_lags, alpha=0.01)
    centered_ci = ci - a[:,None]
    outside = np.abs(a) >= centered_ci[:,1]
    inside = ~outside
    ar_orders = [j for j, x in enumerate(outside) if x]
    #highest_lag = 0
    highest_lag = ar_orders[-1]
    if all(item == 0 for item in ar_orders): #falls nur lag = 0 signifikant (ist immer der Fall) wird nicht getradet 
        trading = False
        lag = 0
    else: #hier wird über das BIC die optimale Lag-Anzahl ausgewertet, alternativ kann man auch das AIC verwenden
        for ar_order in ar_orders:
            model_AR = ARIMA(test_data, order = (ar_order,0,ar_order))
            results_AR = model_AR.fit()
            AIC.append(results_AR.aic)
            BIC.append(results_AR.bic)
        min_value = min(BIC) 
        min_index=BIC.index(min_value)
        lag = ar_orders[min_index]
        
    return lag, trading, highest_lag #gibt den boolschen Parameter "trading" zurück, der bestimmt, ob an Tag X getradet wird (in Kombination mit aduller siehe unten), die optimale Lag-Anzahl ist im Parameter "Lag" gespeichert

In [12]:
#Funktion die prüft ob betrachtetes Zeitfenster stationär ist, gibt entsprechend boolschen Parameter zurück
def adfuller_test(data):
    stationary = False
    results_adfuller = adfuller(data)
    x = results_adfuller[0]
    y = results_adfuller[4]["1%"]
    #Hier wird nun geprüft ob Null-Hypothese der Nicht-Stationarität abgelehnt werden kann
    if x <= y:
        stationary = True
    if x > y:
        stationary = False

    return stationary

In [13]:
#In dieser Funktion findet das eigentliche Trading und die zugehörigen Berechnungen statt
def trade(training, testing, endowment, profit, cash, gewinn, counter_right, counter_general):
    history = [x for x in training["Log. Returns"]]
    real_returns = []
    forecasts = []
    tax_allowance = 801 #Definition der Steuerfreibetragsgrenze

    #In diesem Abschnitt wir der Forecast auf Basis der optimalen Lags der Forecast des ARIMA-Modells bestimmt. Außerdem werden weitere benötigte Parameter bestimmt.
    lag, trading, highest_lag = statistic_tests(history, number_lags)
    model = ARIMA(history, order = (lag,0,lag))
    model_results = model.fit()
    pred = model_results.forecast()
    forecast = pd.DataFrame(pred)
    forecast = forecast[0].iloc[0]
    real_return = testing["Returns"]
    if isinstance(real_return, float) == False:
        real_return = real_return.item()
    forecasts.append(forecast)
    stationary = adfuller_test(history)
    
    prev_endowment = endowment #Variable zur Berechnung der Gewinne aus aktuellem Trade
        
    if (trading == True) and (stationary== True): #getradet wird nur falls sowohl signifikante Ergebnisse aus der PACF erhalten wurden und die Datenreihe stationär ist
        #Counter_general berechnet, wie oft insgesamt getradet wird, counter_right berechnet, wie oft die Trading-Entscheidung korrekt ist
        counter_general = counter_general + 1
        
        # 1.Fall: BUY STOCK
        if forecast >= 0.0 and cash == True:
            #HINWEIS: falls die Broker-Gebühren-Datei vorhanden ist kann "0.0005*endowment" durch "trading_fee(endowment)" ersetzt werden
            endowment = endowment*(real_return/100+1) - 0.0005*endowment #trading_fee(endowment)
            action.append("BUY")
            cash = False
            if real_return >= 0.0:
                counter_right = counter_right + 1
            
        #2. Fall: HOLD STOCK
        elif forecast >= 0.0 and cash == False:
            endowment = endowment*(real_return/100+1) #Vermögen entwickelt sich abhängig von Aktienkursentwicklung
            action.append("HOLD")
            cash = False
            if real_return >= 0.0:
                counter_right = counter_right + 1
            
        #3. Fall: HOLD CASH or SHORT-SELLING
        elif forecast <= 0.0 and cash == True:
            if real_return <= 0.0:
                counter_right = counter_right + 1
            if short == True: #Falls short-selling ausgewählt wurde
                endowment = endowment - (real_return/100)*endowment - 0.0005*endowment #trading_fee(endowment)
                gain = endowment - prev_endowment
                if isinstance(gain, float) == False: #die isinstance-Abfragen (auch die weiter unten) werden nur gemacht weil es bei den folgenden if-Abfragen sonst zu Fehlermeldungen wegen dem zugehörigen Index kommt 
                    gain = gain.item()
                if gain > 0.0:
                    profit = profit + gain
                    #Falls der Gewinn 801 GE noch nicht übersteigt, wird der volle Gewinn aus diesem Trade angerechnet. Andernfalls werden die Kapitalertragssteuern + Soli (und ggf. Kirchensteuer) abgezogen 
                    if profit < 801:
                        endowment = endowment + gain
                    else:
                        endowment = endowment + (1-0.26375)*gain
                action.append("SHORT")
                cash == True
            else: #Falls Short-Selling verneint wurde bleibt Endowment unverändert
                endowment = endowment
                action.append("HOLD")
                cash = True
                
        #4. Fall: SELL STOCK
        elif forecast <= 0.0 and cash == False:
            if real_return <= 0.0:
                counter_right = counter_right + 1
            endowment = endowment - 0.0005*endowment  #trading_fee(endowment)
            gain = endowment - prev_endowment
            if isinstance(gain, float) == False:
                gain = gain.item()
            if gain >= 0:
                profit = profit + gain
                if profit < 801:
                    endowment = endowment + gain
                else:
                    endowment = endowment + (1-0.26375)*gain
            action.append("SELL")
            cash = True
            
    else: #Falls nicht getradet wird:
        if cash == True:
            action.append("No trade today, do nothing")
            endowment = endowment
        #Falls im Voraus getradet wurde, nun aber nicht mehr, werden Aktien verkauft, denn wann sich nächstes signifikantes Zeitfenster ergibt ist ungewiss. 
        #Die Aktien wären so ohne Einfluss des ARIMA-Modells den Schwankungen ausgesetzt. Dies soll verhindert werden. Dieses Vorgehen ist allerdings präferenzabhängig.
        if cash == False:
            action.append("No trade today, sell stocks")
            endowment = endowment - 0.0005*endowment #*(real_return/100+1)
            cash = True
    
    if (isinstance(endowment, float) == False) and (isinstance(endowment, int) == False): #dies dient erneut nur dazu den reinen Endowment-Wert ohne Index zu erhalten, damit Vergleich unten klappt
        endowment = endowment.item()
        
    #Hier wird Gewinnabschöpfung vorgenommen, ebenfalls präferenzabhängig    
    if endowment >= start_endowment:
        gewinn = gewinn + (endowment-start_endowment)
        endowment = 10000.0
    
    return endowment, action, profit, cash, gewinn, counter_right, counter_general

In [15]:
today = date.today()

#Verwendete Aktien:
stocks = ["AAPL", "MSFT", "GOOG", "AMZN", "JNJ", "UNH", "XOM", "V", "TCEHY", "JPM", "MC.PA", "WMT", "TSM", "NVDA", "PG", "LLY", "TSLA", "MA", "CVX", "NESN.SW", "HD", "ABBV", "META", "MRK", "BRK-B", "PFE", "BAC", "BABA", "KO", "ROG.SW", "PEP", "AVGO", "NVO", "ASML", "ORCL", "TM", "NOVN.SW", "AZN", "COST", "TMO", "BP", "OR.PA", "SHEL", "CSCO", "MCD", "ABT", "TMUS", "DHR", "RMS.PA", "ACN", "VZ", "DIS", "NEE", "BHP", "TTE", "CMCSA", "WFC", "PM", "TXN", "NKE", "LIN.DE", "BMY", "0939.HK", "SCHW", "RTX", "MS", "COP", "AMGN", "CDI.PA", "HON", "CRM", "NFLX", "0941.HK", "T", "HSBC", "RY", "SAP", "UPS", "3690.HK", "IBM", "CAT", "UL", "QCOM", "BA", "LMT", "SIE.DE", "SAN.PA", "LOW", "CVS", "CBA.AX", "INTC", "BMW.DE", "GS", "LHA.DE", "MBG.DE", "AMCR", "MAR", "BAYN.DE", "RWE.DE", "PAH3.DE"]

endowment_after_trade = [] 
gewinn_liste = []  #Liste dient zur Speicherung der Gewinne der einzelnen Aktien
number_stocks = len(stocks)
#counter = 0
#virtual_today = datetime(2021, 4, 2)  #diese Variable dient zur Durchführung eines Testlaufes in der Vergangenheit (nicht bis zum aktuellen Tag sondern bis zu einem fiktiven heutigen Tag). Am Start der while-Schleife muss dementsprechdend today durch virtual_today ersetzt werden
#virtual_today = virtual_today.date()

#Counter, welche die Genauigkeit des Modells bestimmen
counter_right = 0
counter_general = 0

for stock in stocks:
    action = []  #dient zur Speicherung und Ausgabe der durchgeführten Handlungen mit der jeweiligen Aktie
    gewinn = 0  #gibt den Gewinn an, der mit jeweiliger Aktie erzielt wurde. Endwert wird an "gewinn_liste" übergeben
    gewinn_liste_aktie = [] #zeigt den Gewinnverlauf einer einzelnen Aktie an
    endowment_verlauf_aktie = []  #zeigt den Verlauf des Vermögens während des Tradings an (kann nur <= start_endowment sein da Gewinn ja abgeschöpft wird)  
    profit = 0
    min_start = datetime(2022, 6, 21)
    min_start = min_start.date()
    real_start = datetime(2022, 11, 4) #bildet zusammen mit "end" erstes Intervall welches auf Signifikanz geprüft wird, wird dann in jeder Iteration um einen Tag in die Vergangenheit verändert (bis min_start erreicht), bis signifikantes Zeitfenster gefunden
    real_start = real_start.date()
    end = datetime(2022, 12, 15) #gibt nur das ERSTE Enddatum an, wird sukzessive um einen Tag erhöht bis man bei "today" (oder "virtual_today" (hypothetisches Enddatum des Tests)) angekommen ist 
    end = end.date()
    print(stock)
    endowment = 10000.0  #Wie viel Geld soll in jeweilige Aktie investiert werden
    start_endowment = endowment
    cash = True #Zu Beginn startet man jeweils mit Bargeld

    data_1 = get_data(stock, min_start, today+timedelta(days=1))  #Download der Daten der Aktie für das komplette Zeitfenster

    while end != today:
        
        print(end)
        trading = False
        
        number_lags = 1
        counter = 0 #Hilfsvariable für Lag-Erhöhung
        real_start_change = real_start
        len_before = 0
        
        if np.is_busday(end) == True:  #Trading nur an Börsentagen
            while (trading == False) and (real_start_change != min_start):
                data = data_1[(data_1["Date"] >= str(real_start_change)) & (data_1["Date"]<= str(end+timedelta(days=1)))]
                df_train, df_test = split_one_day(data) #wird nicht zwingend benötigt
                lag, trading, highest_lag = statistic_tests(df_train["Log. Returns"], number_lags)
                real_start_change = real_start_change - timedelta(days=1)
                counter = counter + 1
                if number_lags < 11: #Maximalanzahl der zu prüfenden Lags 
                    #sukzessive Erhöhung der zu prüfenden Lags bei vergrößertem Zeitraum
                    if (len(data) % 5 == 0) and len_before != len(data):
                        number_lags = number_lags + 1
                len_before = len(data)
            
            if trading == True:
                df_train, df_test = split_one_day(data)
                endowment, action, profit, cash, gewinn, counter_right, counter_general = trade(df_train, df_test, endowment, profit, cash, gewinn, counter_right, counter_general)
            
            if trading == False:  #hier wird noch das jeweils größtmögliche Zeitfenster auf Signifikanz getestet. Falls sich doch noch Signifikanz ergibt wird getradet.
                df_train, df_test = split_one_day(data)
                endowment, action, profit, cash, gewinn, counter_right, counter_general = trade(df_train, df_test, endowment, profit, cash, gewinn, counter_right, counter_general)
                
            print(action)
            
            if endowment <= start_endowment*0.93:
                break

            gewinn_liste_aktie.append(gewinn)
            endowment_verlauf_aktie.append(endowment)
            
        else:
            print("No trading day (Weekend etc.)")
    
        #Verschiebung des kompletten Zeitfensters, damit anfängliche Zeitfenstergrößen unverändert bleiben
        min_start = min_start + timedelta(days=1)
        real_start = real_start + timedelta(days=1)
        end = end + timedelta(days=1)
        
        print(endowment)
    
    print(endowment)
    endowment_after_trade.append(endowment)
    gewinn_liste.append(gewinn)

print(endowment_after_trade)

AAPL
[*********************100%***********************]  1 of 1 completed
2022-12-15
['No trade today, do nothing']
10000.0
2022-12-16
['No trade today, do nothing', 'No trade today, do nothing']
10000.0
2022-12-17
No trading day (Weekend etc.)
10000.0
2022-12-18
No trading day (Weekend etc.)
10000.0
2022-12-19
['No trade today, do nothing', 'No trade today, do nothing', 'No trade today, do nothing']
10000.0
2022-12-20
['No trade today, do nothing', 'No trade today, do nothing', 'No trade today, do nothing', 'No trade today, do nothing']
10000.0
2022-12-21
['No trade today, do nothing', 'No trade today, do nothing', 'No trade today, do nothing', 'No trade today, do nothing', 'No trade today, do nothing']
10000.0
2022-12-22
['No trade today, do nothing', 'No trade today, do nothing', 'No trade today, do nothing', 'No trade today, do nothing', 'No trade today, do nothing', 'No trade today, do nothing']
10000.0
2022-12-23
['No trade today, do nothing', 'No trade today, do nothing', 'No tr

KeyboardInterrupt: 

In [ ]:
#Darstellung der Ergebnisse
print(sum(endowment_after_trade))
print(len(stocks)*start_endowment)
print(sum(gewinn_liste))
ergebnis = sum(endowment_after_trade)+ sum(gewinn_liste)
print(ergebnis)

if (len(stocks)*start_endowment) < ergebnis:
    print("Im ausgewählten Zeitraum wurde mit den ausgewählten Aktien ein Gewinn erwirtschaftet!")